## language model using a bert model
* take 1

* Try 1 : keras based [bert-for-tf2], https://github.com/kpe/bert-for-tf2 

In [2]:
# !pip install sentencepiece
# ! pip install pytorch_transformers

  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp37-none-any.whl size=884004 sha256=649cc9b2ce307eb2f82f4ebe05e7fd115a57f9d5793cdc9fcecea4fb70dec4f5
  Stored in directory: C:\Users\Dan Ofer\AppData\Local\pip\Cache\wheels\63\2a\db\63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built sacremoses


In [13]:
import subprocess
# import keras

import sentencepiece as spm

%matplotlib inline

In [14]:
sequences_file = "./data/sample_seqs.csv"

In [5]:
import pandas as pd

proteins = pd.read_csv('data/proteins.csv.gz')
proteins["seq_len"] = proteins["seq"].str.len()
print(proteins.shape)
proteins.head()

(1000000, 5)


,tax_id,uniprot_name,seq,n_complete_go_annotations,complete_go_annotation_indices
0,1572043.0,A0A5A9P0L4_9TELE,MEEITQIKKRLSQTVRLEGKEDLLSKKDSITNLKTEEHVSVKKMVI...,0,[]
1,1930593.0,A0A410P257_9BACT,MKAIAWLIVLTFLPEQVAWAVDYNLRGALHGAVAPLVSAATVATDG...,0,[]
2,72004.0,A0A5A9Q4C1_9CETA,MSSQESPAVEFSTTTVSSVAVQAGDSKIVIAVIKCGKWIQLQLAES...,0,[]
3,375764.0,UPI0011814A49,MSTQAPSFTQPLQSVVALEGSAATFQAQVSGSPVPEVSWFRDGQVL...,0,[]
4,1415580.0,UPI001186A1A9,MTTEAPKFLQPLQSVVALEGSAATFEARISGFPVPEVSWFRDGQVL...,0,[]


In [8]:
proteins.head()

,tax_id,uniprot_name,seq,n_complete_go_annotations,complete_go_annotation_indices,seq_len
0,1572043.0,A0A5A9P0L4_9TELE,MEEITQIKKRLSQTVRLEGKEDLLSKKDSITNLKTEEHVSVKKMVI...,0,[],45354
1,1930593.0,A0A410P257_9BACT,MKAIAWLIVLTFLPEQVAWAVDYNLRGALHGAVAPLVSAATVATDG...,0,[],39677
2,72004.0,A0A5A9Q4C1_9CETA,MSSQESPAVEFSTTTVSSVAVQAGDSKIVIAVIKCGKWIQLQLAES...,0,[],38354
3,375764.0,UPI0011814A49,MSTQAPSFTQPLQSVVALEGSAATFQAQVSGSPVPEVSWFRDGQVL...,0,[],38182
4,1415580.0,UPI001186A1A9,MTTEAPKFLQPLQSVVALEGSAATFEARISGFPVPEVSWFRDGQVL...,0,[],38163


In [9]:
proteins.describe()

,tax_id,n_complete_go_annotations,seq_len
count,9.999770e+05,1000000.000000,1000000.000000
mean,5.562390e+05,9.067817,2336.231343
std,7.443289e+05,17.448869,1427.527815
min,2.000000e+00,0.000000,11.000000
25%,1.014100e+04,0.000000,1687.000000
50%,1.258780e+05,0.000000,1958.000000
75%,1.037660e+06,12.000000,2486.000000
max,2.651628e+06,632.000000,45354.000000


In [10]:
proteins.loc[proteins["seq_len"]<4000].shape[0]

(928988, 6)

### output sample of sequences for LM training
* We run the sentence piece on this beforehand in the other notebok - sentencepiece 
* http://localhost:8888/notebooks/toyData_ProBERT.ipynb

In [12]:

proteins.loc[proteins["seq_len"]<5000].sample(n=30000)["seq"].to_csv("./data/sample_seqs.csv", index = False)

C:\Users\Dan Ofer\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [ ]:
# # spm.SentencePieceTrainer.Train('--input=uniclust30_2016_1M_sampledSeq.fasta --model_prefix=m --vocab_size=1500')
# spm.SentencePieceTrainer.Train('--input=%s --model_prefix=m --vocab_size=16000'%sequences_file)

## tokenize using opreviuously trained sentence pioece

In [15]:
# makes segmenter instance and loads the model file (m.model)
sp = spm.SentencePieceProcessor()
sp.load('m.model')

True

In [16]:
eg_seq = "MRYTVLIALQGALLLLLLIDDGQGQSPYPYPGMPCNSSRQCGLGTCVHSRCAHCSSDGTLCSPEDPTMVWPCCPESSCQLVVGLPSLVNHYNCLPNQCTDSSQCPGGFGCMTRRSKCELCKADGEACNSPYLDWRKDKECCSGYCHTEARGLEGVCIDPKKIFCTPKNPWQLAPYPPSYHQPTTLRPPTSLYDSWLMSGFLVKSTTAPSTQEEEDDY"
print(sp.encode_as_pieces(eg_seq))
print(sp.encode_as_ids(eg_seq))

['▁', 'MR', 'YTV', 'LIA', 'L', 'QGA', 'L', 'LLLLL', 'IDD', 'GQGQ', 'SPY', 'PYP', 'GMP', 'CNS', 'SRQ', 'CGL', 'GTC', 'VHS', 'RCA', 'HCS', 'SDGT', 'LCS', 'PED', 'PT', 'MV', 'WP', 'CCP', 'ES', 'SCQ', 'L', 'VVG', 'LPSL', 'VNH', 'YNC', 'LPN', 'QCT', 'DSS', 'QCP', 'GGFG', 'CM', 'TRR', 'SK', 'CELC', 'KAD', 'GEA', 'CNS', 'PYL', 'DWR', 'KDKE', 'CCS', 'GY', 'CH', 'TEA', 'RGLE', 'GVC', 'IDP', 'KKI', 'FCT', 'PKNP', 'WQ', 'LAP', 'YPP', 'SYH', 'QPT', 'TLR', 'PPTS', 'LYD', 'SWL', 'M', 'SGF', 'LVK', 'STTA', 'PST', 'QEEE', 'DDY']
[0, 261, 3435, 2112, 3, 2866, 3, 7330, 1389, 9500, 3091, 4060, 5365, 6116, 2653, 5790, 5014, 3522, 6044, 6251, 7308, 3364, 1347, 165, 227, 606, 6383, 76, 5146, 3, 917, 8768, 6529, 11718, 2114, 8040, 498, 8400, 8666, 683, 2705, 70, 11787, 3365, 1052, 6116, 3694, 6163, 8661, 6444, 255, 503, 1356, 14930, 3704, 2768, 1160, 9902, 15766, 596, 1937, 6559, 6055, 2673, 989, 7826, 3396, 4156, 9, 2157, 1091, 7020, 618, 7252, 3213]


In [26]:
sample_proteins = proteins[proteins['seq_len'] < 500].sample(1000).copy()
sample_proteins['seq_token_ids'] = sample_proteins['seq'].apply(sp.encode_as_ids)
sample_proteins['seq_n_tokens'] = sample_proteins['seq_token_ids'].apply(len)
sample_proteins.head()

,tax_id,uniprot_name,seq,n_complete_go_annotations,complete_go_annotation_indices,seq_len,seq_token_ids,seq_n_tokens
613736,175836.0,A0A091GX50_BUCRH,STGREGEMKILGHCSSQVTSTLGEFELEGICRLQDQASPSDSEPRA...,0,[],212,"[0, 1237, 1820, 4968, 1841, 6251, 2208, 549, 3...",74
268740,1773.0,A0A4U5KFK5_MYCTX,YSLPAIPLGIDVGGAIGPLTTPPITIPAIPLGIDVSGSLGPINIPI...,0,[],441,"[0, 1927, 15625, 3, 1936, 8213, 4216, 3, 706, ...",142
878089,9669.0,M1EBI8_MUSPF,GYASSLVFLTYVISFIFRNRRKNSGIWSFFFFIITIFTIVATDINE...,0,[],206,"[0, 4180, 10368, 2131, 5140, 1791, 3925, 1702,...",72
395371,1165861.0,A0A0L0UHW2_9BASI,MDRASKKSEQFIIPEPDSPFPFNPIYHGLSGPAANSFPKYVPPQFK...,0,[],137,"[0, 3909, 2050, 1480, 4199, 2170, 7424, 4659, ...",45
820211,32507.0,A0A3Q4G7M5_NEOBR,MTSLAHQLKRLALPQSDANLLTRREVASLLFDPKEAASMDRSTFYA...,0,[],120,"[0, 2854, 3, 4650, 3, 11793, 1462, 1326, 13990...",40


In [30]:
display(sample_proteins['seq_len'].describe())
display(sample_proteins['seq_n_tokens'].describe())


count    1000.000000
mean      202.234000
std       123.001417
min        11.000000
25%       108.000000
50%       167.500000
75%       289.000000
max       499.000000
Name: seq_len, dtype: float64

count    1000.000000
mean       66.897000
std        39.755187
min         4.000000
25%        36.000000
50%        56.000000
75%        96.000000
max       166.000000
Name: seq_n_tokens, dtype: float64

# Let's BERT!

In [31]:
from bert import BertModelLayer

In [32]:
l_bert = BertModelLayer(**BertModelLayer.Params(
  vocab_size               = 16000,        # embedding params
  use_token_type           = True,
  use_position_embeddings  = True,
  token_type_vocab_size    = 2,

  num_layers               = 12,           # transformer encoder params
  hidden_size              = 768,
  hidden_dropout           = 0.1,
  intermediate_size        = 4*768,
  intermediate_activation  = "gelu",

  adapter_size             = None,         # see arXiv:1902.00751 (adapter-BERT)

  shared_layer             = False,        # True for ALBERT (arXiv:1909.11942)
  embedding_size           = None,         # None for BERT, wordpiece embedding size for ALBERT

  name                     = "bert"        # any other Keras layer params
))

In [35]:
from tensorflow import keras

max_seq_len = 166
l_input_ids = keras.layers.Input(shape=(max_seq_len,), dtype='int32')
output = l_bert(l_input_ids) # output: [batch_size, max_seq_len, hidden_size]
model = keras.Model(inputs=l_input_ids, outputs=output)
model.build(input_shape=(None, max_seq_len))

TypeError: in converted code:
    relative to C:\Users\Dan Ofer\Anaconda3\lib\site-packages:

    bert\model.py:74 call
        output           = self.encoders_layer(embedding_output, mask=mask, training=training)
    tensorflow\python\keras\engine\base_layer.py:591 __call__
        self._maybe_build(inputs)
    tensorflow\python\keras\engine\base_layer.py:1881 _maybe_build
        self.build(input_shapes)
    bert\transformer.py:214 build
        name="layer_{}".format(layer_ndx),
    params_flow\layer.py:79 from_params
        **kwargs                                            # override with kwargs
    params_flow\layer.py:30 __init__
        self._construct(self.params)
    bert\transformer.py:137 _construct
        if params.hidden_size % params.num_heads != 0:

    TypeError: unsupported operand type(s) for %: 'int' and 'NoneType'
